## Dog Cat Classification

### 1 - Package import 

In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing
import matplotlib.pyplot as plt
import os
import random

import tensorflow as tf
import tensorflow.keras.layers as tfl

from keras.preprocessing.image import ImageDataGenerator, load_img
from tensorflow.keras.utils import to_categorical

from sklearn.model_selection import train_test_split

import warnings
warnings.filterwarnings('ignore')

In [2]:
from numpy.random import seed
seed(1)

tf.random.set_seed(2)

### 2 - Data Import

In [3]:
train_path = '../input/dogs-vs-cats/train.zip'
test_path = '../input/dogs-vs-cats/test1.zip'

destination = '/kaggle/files/images'

from zipfile import ZipFile as zipper
with zipper(train_path, 'r') as zipp:
    zipp.extractall(destination)
    
with zipper(test_path, 'r') as zipp:
    zipp.extractall(destination)

In [4]:
train = pd.DataFrame({'file': os.listdir('/kaggle/files/images/train')})

categories = []
for i in os.listdir('/kaggle/files/images/train'):
    if 'dog' in i:
        categories.append(1)
    else:
        categories.append(0)
        
train['categories'] = categories

test = pd.DataFrame({'file': os.listdir('/kaggle/files/images/test1')})

### 3 - See an example 

In [5]:
def example_im(index):
    '''
    Function to display an example image.
    
    index -- which image in the training set. 
    '''
    im = plt.imread('/kaggle/files/images/train/'+str(train['file'][index]))

    print(type(im))
    print(im.shape)
    print(type(im.shape))
    
    plt.imshow(im)
    plt.axis('off')
    
example_im(1)

### 4 - Data Preparation 

In [6]:
train['categories'].value_counts()

For our data, we have same number of cats and dogs and training data.

In [7]:
train.shape

The numeric varaible in `categories` columns is mapped to string for later image generation

The training data is not a particularly small dataset so we can use `10%` of its data as a `cross validation set`, and `random_state` is set to be 0 for reproductivity.

In [8]:
train['categories'] = train['categories'].replace({0: 'cat', 1: 'dog'})
train_set, val_set = train_test_split(train, test_size=0.1, random_state = 0)

In [9]:
train_gen_scaled = ImageDataGenerator(rescale=1./255)
val_gen_scaled = ImageDataGenerator(rescale=1./255)

batch_size = 64

train_generator_scaled = train_gen_scaled.flow_from_dataframe(
    dataframe = train_set,
    directory = destination + '/train/', # file path format
    x_col = 'file',
    y_col = 'categories',
    class_mode = 'categorical',
    target_size = (224,224),
    batch_size = batch_size
)


validation_generator_scaled = val_gen_scaled.flow_from_dataframe(
    dataframe = val_set,
    directory = destination + '/train/',
    x_col = 'file',
    y_col = 'categories',
    class_mode = 'categorical',
    target_size = (224,224),
    batch_size = batch_size
)


In [10]:
def mini_batch_example_plot(df):

    example_generator = train_gen_scaled.flow_from_dataframe(
        dataframe = df,
        directory = destination + '/train/',
        x_col = 'file',
        y_col = 'categories',
        class_mode = 'categorical',
        target_size = (224,224)
    )
    
    fig, ax  = plt.subplots(2,4,figsize=(12, 12))
    ax = ax.flatten()
    
    for i in range(8):
        X, Y = next(example_generator)
        image = X[0]
        ax[i].imshow(image)
        ax[i].axis('off')
    
    
mini_batch_example_plot(train_set)

### 5 - CNN model construction

In [11]:
from tensorflow.keras.layers import (
    BatchNormalization, Conv2D, MaxPooling2D, Activation, Flatten, Dropout, Dense
)

from tensorflow.keras.layers.experimental.preprocessing import RandomFlip, RandomRotation


In [12]:
# set constants
(IMAGE_WIDTH, IMAGE_HEIGHT, IMAGE_CHANNELS) = (224,224,3)
IMAGE_SIZE  = (IMAGE_WIDTH, IMAGE_HEIGHT)
IMAGE_SHAPE = (224,224,3)

In [13]:
def shallow_CNN_Model(image_shape, augmentation = False):
    '''
    This creates a shallow CNN model, the structure uses the idea from VGG-16
    '''
    if (augmentation == True):
        model = tf.keras.Sequential([RandomFlip("horizontal",input_shape = image_shape),
                   RandomRotation(0.1)])
    else: 
        model = tf.keras.Sequential()

    model.add(Conv2D(32, (3, 3), activation='relu', input_shape = image_shape))
    model.add(Conv2D(32, (3, 3), activation='relu'))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))

    model.add(Conv2D(64, (3, 3), activation='relu'))
    model.add(Conv2D(64, (3, 3), activation='relu'))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))

    model.add(Conv2D(128, (3, 3), activation='relu'))
    model.add(Conv2D(128, (3, 3), activation='relu'))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))

    model.add(Flatten())
    model.add(Dense(512, activation='relu'))
    model.add(BatchNormalization())
    model.add(Dropout(0.5))
    model.add(Dense(2, activation='softmax'))

    return model

In [14]:
model = shallow_CNN_Model(IMAGE_SHAPE)
model_augmented = shallow_CNN_Model(IMAGE_SHAPE,True)

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model_augmented.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

model.summary()

### 6 - Early stopping after seeing no decrease in loss over 5 epoches

In [15]:
from tensorflow.keras.callbacks import EarlyStopping

earlystop = EarlyStopping(patience=5)

In [16]:
def model_fitting(model, train_generator, val_generator, callbacks, epochs):
    '''
    This is a function used to fit generator, with customized input 
    model, callbacks, and epochs. 
    '''
    return model.fit_generator(
        train_generator, 
        epochs = epochs,
        validation_data = val_generator,
        validation_steps = val_set.shape[0]//64,
        steps_per_epoch = train_set.shape[0]//64,
        callbacks = callbacks 
    )

    

In previous test, the dataset not being augmented performs worse than augmented dataset on validation set, showing a sign of overfitting in training set.  

In [17]:
history_epoch50 = model_fitting(model,train_generator_scaled, validation_generator_scaled, [earlystop], epochs = 50)

In [18]:
history_epoch50_augmented = model_fitting(model_augmented,train_generator_scaled, validation_generator_scaled, [earlystop], epochs = 50)

In [19]:
model_augmented.save_weights('model_augmented_weights.h5')

In [20]:

def plotting_loss(history,upper_bound):

    fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(10, 4))
    
    ax1.plot(history.history['loss'], color='b', label="Training loss")
    ax1.plot(history.history['val_loss'], color='r', label="validation loss")
    ax1.set_xticks(np.arange(1, upper_bound, 1))
    ax1.set_yticks(np.arange(0, 1, 0.1))

    ax2.plot(history.history['accuracy'], color='b', label="Training accuracy")
    ax2.plot(history.history['val_accuracy'], color='r',label="Validation accuracy")
    ax2.set_xticks(np.arange(1, upper_bound, 1))

    legend = plt.legend(loc='best', shadow=True)
    plt.tight_layout()
    plt.show()


In [21]:
plotting_loss(history_epoch50, 50)

In [22]:
plotting_loss(history_epoch50_augmented, 40)

Using functional API for easier operation on last few layers for transfer learning.

In [23]:
def data_augmenter():
    '''
    Create a Sequential model composed of 2 layers
    Returns:
        tf.keras.Sequential
    '''
    ### START CODE HERE
    data_augmentation = tf.keras.Sequential()
    data_augmentation.add(RandomFlip('horizontal'))
    data_augmentation.add(RandomRotation(0.2))
    ### END CODE HERE
    
    return data_augmentation

In [24]:
train_gen = ImageDataGenerator()
val_gen = ImageDataGenerator()
batch_size = 64

train_generator = train_gen.flow_from_dataframe(
    dataframe = train_set,
    directory = destination + '/train/', # file path format
    x_col = 'file',
    y_col = 'categories',
    class_mode = 'categorical',
    target_size = (224,224),
    batch_size = batch_size
)


validation_generator = val_gen.flow_from_dataframe(
    dataframe = val_set,
    directory = destination + '/train/',
    x_col = 'file',
    y_col = 'categories',
    class_mode = 'categorical',
    target_size = (224,224),
    batch_size = batch_size
)

In [25]:
input_shape = (224,224, 3)

def transfer_learning(model,image_shape=input_shape, data_augmentation=data_augmenter()):
    ''' Define a tf.keras model for binary classification out of the MobileNetV2 model
    Arguments:
        image_shape -- Image width and height
        data_augmentation -- data augmentation function

    Returns:
        tf.keras.model
    '''
    if model == 'MobileNetV2':
        base_model = tf.keras.applications.MobileNetV2(input_shape=input_shape,
                                                       include_top=False, 
                                                       weights='imagenet') # From imageNet
        preprocess_input = tf.keras.applications.mobilenet_v2.preprocess_input
        
        
    if model == 'VGG':
        base_model = tf.keras.applications.VGG16(input_shape=input_shape,
                                                    weights="imagenet", 
                                                    include_top=False)
        preprocess_input = tf.keras.applications.vgg16.preprocess_input
        
        
    # freeze the base model by making it non trainable
    base_model.trainable = False

    # create the input layer (Same as the imageNetv2 input size)
    inputs = tf.keras.Input(shape = input_shape) 
    
    # apply data augmentation to the inputs
    x = data_augmentation(inputs)
    
    # data preprocessing using the same weights the model was trained on
    x = preprocess_input(x) 
    
    # set training to False to avoid keeping track of statistics in the batch norm layer
    x = base_model(x, training=False) 
    
    # add the new Binary classification layers
    # use global avg pooling to summarize the info in each channel
    x = tfl.GlobalAveragePooling2D()(x) 
    # include dropout with probability of 0.2 to avoid overfitting
    x = tfl.Dropout(0.2)(x)
        
    # use a prediction layer with one neuron (as a binary classifier only needs one)
    outputs = tfl.Dense(units = 2)(x)

    new_model = tf.keras.Model(inputs, outputs)
    
    return new_model


In [26]:
model_trans_VGG = transfer_learning('VGG',(224,224,3), data_augmenter())
base_learning_rate = 0.001
model_trans_VGG.compile(
              #optimizer=tf.keras.optimizers.Adam(lr=base_learning_rate),
              optimizer = 'adam',
              loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              metrics=['accuracy'])

model_trans_Mobile = transfer_learning('MobileNetV2',(224,224,3), data_augmenter())
base_learning_rate = 0.001
model_trans_Mobile.compile(
                #optimizer=tf.keras.optimizers.Adam(lr=base_learning_rate),
                optimizer = 'adam',
                loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
                metrics=['accuracy'])

model_trans_VGG.summary()

In [27]:
model_trans_Mobile.summary()

In [28]:
history_trans_VGG = model_fitting(model_trans_VGG,train_generator, validation_generator, [earlystop], epochs = 20)

In [29]:
history_trans_Mobile = model_fitting(model_trans_Mobile,train_generator, validation_generator, [earlystop], epochs = 20)

In [30]:
plotting_loss(history_trans_VGG, 20)

In [31]:
plotting_loss(history_trans_Mobile, 20)